In [24]:
try:
    if is_init:
        pass
except:
    %pwd
    %cd ..
    %pwd
    is_init = True

In [25]:
from pathlib import Path
from src.utils import FileUtils
from config import Config
from utils import rmse
import torch
import numpy as np
import pandas as pd
import altair as alt
import random
import itertools
data_folder = '../local_data'
work_folder = '../local_work'
configs_folder = '../configs'

In [26]:
def denormalize_decode_result(y_hat, max_speed, max_spacing, min_speed, min_spacing):
    """input: 1,14 tensor
        output: 1,3 tensor with wattage no longer one hot encoded"""
    watts = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]

    watt_arg = torch.argmax(y_hat[0][2:])
    y_final = torch.empty(3, dtype=torch.float32)
    y_final[0] = max_speed * y_hat[0][0] # TODO call the scale
    y_final[1] = max_spacing * y_hat[0][1] # TODO call the scale
    y_final[2]= watts[watt_arg]
    return y_final

# Inverse

In [27]:
filepath = Path(f"{work_folder}/saved_best/best_I-resnet1d-stainless-2022-07-11_01-00.ckpt") #CHANGEME
print(filepath)
if not Path.is_file(filepath):
    raise Exception(f'Model file does not exist at {filepath}!')

../local_work/saved_best/best_I-resnet1d-stainless-2022-07-11_01-00.ckpt


In [28]:
def get_test_data(path):
    stain_data = torch.load(
                    Path(f"{data_folder}/{path}"))#["laser_params"] #stainless-steel-revised-shuffled inconel-revised-raw-shuffled
    wave_data = stain_data["interpolated_wavelength"]
    emiss_data = stain_data["interpolated_emissivity"]#.keys()#["wavelenght"]
    #print(wave_test.max(1), wave_test.min(1)) #2.5002, 11.9479
    #print(emiss_test.max(1)[0])#.max(1), emiss_test.min(1)) #0, 1
    wave_test = wave_data[round(len(wave_data) * .95):]  
    emiss_test = emiss_data[round(len(wave_data) * .95):]
    return wave_test, emiss_test
#print(max_wave, min_wave)

In [29]:
#pass test data into inverse model, get predicted parameters
from src.models import InverseModel
i_model = InverseModel.load_from_checkpoint(filepath, strict=False)
i_model.eval()

wave_test, emiss_test = get_test_data('stainless-steel-revised-shuffled.pt')
test_y_hats = []#torch.empty(size = len(emiss_test))
for emiss in emiss_test:
    with torch.no_grad():
        test_y_hats.append(i_model(emiss.reshape(1, 800)))

/home/spencersong/anaconda3/envs/meta/lib/python3.10/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/home/spencersong/anaconda3/envs/meta/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:217: UserWarning: Found keys that are not in the model state dict but in the checkpoint: ['direct_model.model.0.weight', 'direct_model.model.0.bias', 'direct_model.model.2.layers.0.block.0.weight', 'direct_model.model.2.layers.0.block.0.bias', 'direct_model.model.2.layers.1.block.0.weight', 'direct_model.model.2.layers.1.block.0.bias', 'direct_model.model.2.layers.2.block.0.weight', 'direct_model.model.2.layers.2.block.0.bias', 'direct_model.model.2.layers.3.block.0.weight', 'direct_model.model.2.layers.3.block.0.bias', 'direct_model.model.2.layers.4.block.0.weight', 'direct_model.model.2.

In [30]:
test_y_hats[0]

tensor([[0.1704, 0.3627, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])

# Direct Model

In [32]:
from src.models import DirectModel
direct_filepath = Path(""
    f"{work_folder}/saved_best/best_D-ann-inconel-2022-07-05_14-39.ckpt") #CHANGEME
print(direct_filepath)
if not Path.is_file(direct_filepath):
    raise Exception(f'Model file does not exist at {direct_filepath}!')

../local_work/saved_best/best_D-ann-inconel-2022-07-05_14-39.ckpt


In [35]:
d_model = DirectModel.load_from_checkpoint(direct_filepath)
d_model.eval()
emiss_y_hats = []
for x in test_y_hats:
    with torch.no_grad():
        emiss_y_hats.append(d_model(x.reshape(1,14)))

In [43]:
emiss_y_hats[1].flatten().tolist()

AttributeError: 'list' object has no attribute 'item'

In [44]:
#calculate rmse
pred_rmse_vals = []
columns = ["wavelength", "emissivity", "prediction", "example_number"]
viz_coords = pd.DataFrame(columns = columns)
for i in range(len(emiss_test)):
    pred_rmse_vals.append(rmse(emiss_y_hats[i], emiss_test[i].flatten()).item())
    # for j in range(len(wave_test[i].tolist())):
    #     viz_coords.append([wave_test[i].tolist()[j], emiss_test[i].tolist()[j], "True", i])
    #     viz_coords.append([wave_test[i].tolist()[j], emiss_y_hats[i].flatten().tolist()[j], "Prediction", i])
mean_rmse = np.mean(pred_rmse_vals)
stdev = np.std(pred_rmse_vals)
print(f'Direct Filepath: {direct_filepath}')
print(f'Inverse Filepath: {filepath}')
print(f'SD of RMSEs on test data: {stdev}')
print(f'Mean of RMSEs on test data: {mean_rmse}')


Direct Filepath: ../local_work/saved_best/best_D-ann-inconel-2022-07-05_14-39.ckpt
Inverse Filepath: ../local_work/saved_best/best_I-resnet1d-stainless-2022-07-11_01-00.ckpt
SD of RMSEs on test data: 0.17008276292283978
Mean of RMSEs on test data: 0.18374059075848925


/home/spencersong/metamaterials_ai/src/utils/utils.py:43: UserWarning: Using a target size (torch.Size([800])) that is different to the input size (torch.Size([1, 800])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return (torch.nn.functional.mse_loss(pred, target) + epsilon).sqrt()


In [46]:
#histogram
hist_input = pd.DataFrame({"RMSE":pred_rmse_vals})
alt.Chart(hist_input).mark_bar().encode(
    alt.X("RMSE:Q", bin=True),
    y='count()',
).properties(title= 'Test RMSE Scores')


alt.Chart(...)

In [124]:
columns = ["wavelength(um)", "emissivity", "prediction", "example_number", ]
viz_coords = pd.DataFrame(columns = columns)
iters = 2 #len(emiss_test)):
for i in range(9):
    true_list = [wave_test[i].tolist(), emiss_test[i].tolist(), ["True" for x in range(800)], [f'Ex:{i}, RMSE:{np.round(pred_rmse_vals[x], decimals=3)}' for x in range(800)]]
    true_list = list(map(list, zip(*true_list)))
    pred_list = [wave_test[i].tolist(), emiss_y_hats[i].flatten().tolist(), ["Pred" for x in range(800)], [f'Ex:{i}, RMSE:{np.round(pred_rmse_vals[x], decimals=3)}' for x in range(800)]]
    pred_list = list(map(list, zip(*pred_list)))
    viz_coords = viz_coords.append(pd.DataFrame(true_list, columns = columns))
    viz_coords = viz_coords.append(pd.DataFrame(pred_list, columns = columns))

/tmp/ipykernel_2963087/2435539032.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  viz_coords = viz_coords.append(pd.DataFrame(true_list, columns = columns))
/tmp/ipykernel_2963087/2435539032.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  viz_coords = viz_coords.append(pd.DataFrame(pred_list, columns = columns))
/tmp/ipykernel_2963087/2435539032.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  viz_coords = viz_coords.append(pd.DataFrame(true_list, columns = columns))
/tmp/ipykernel_2963087/2435539032.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  viz_coords = viz_coords.append(pd.DataFrame(pred_list, columns = columns)

In [125]:
viz_coords

,wavelength(um),emissivity,prediction,example_number
0,2.500184,0.498524,True,"Ex:0, RMSE:0.062"
1,2.512008,0.496808,True,"Ex:0, RMSE:0.101"
2,2.523833,0.49548,True,"Ex:0, RMSE:0.099"
3,2.535657,0.494429,True,"Ex:0, RMSE:0.162"
4,2.547481,0.493368,True,"Ex:0, RMSE:0.042"
...,...,...,...,...
795,11.900584,0.123691,Pred,"Ex:8, RMSE:0.246"
796,11.912409,0.143161,Pred,"Ex:8, RMSE:0.098"
797,11.924233,0.117372,Pred,"Ex:8, RMSE:0.187"
798,11.936058,0.12246,Pred,"Ex:8, RMSE:0.093"


In [130]:
#plot differences between predictions and actual!
alt.data_transformers.disable_max_rows()

chart = alt.Chart(viz_coords).mark_line().encode(
    x='wavelength(um)',
    y='emissivity',
    color='prediction'
).facet(facet = 'example_number')#, columns = 3)
chart.display()

alt.FacetChart(...)

In [76]:
#visualize predictions vs original
